In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np

In [2]:
#read data from conpressed file
vehicle_data = pd.read_csv("vehicle_data.csv.gz",
               compression="gzip")

#strip extra spaces in field
vehicle_data['fuel_type'].str.rstrip().astype(str)
vehicle_data['elec_hybrid']= vehicle_data['fuel_type'].isin(['ELECTRIC','NONE','OTHER'])
vehicle_data


,vin,registration_class,zip,model_year,body_type,fuel_type,elec_hybrid
0,999407G3573,PAS,12866,1976,SUBN,GAS,False
1,9992313,PAS,13316,1924,CONV,GAS,False
2,998867,PAS,11501,1952,2DSD,GAS,False
3,9983692,PAS,10940,1936,2DSD,GAS,False
4,99782010525,PAS,11225,1978,2DSD,GAS,False
...,...,...,...,...,...,...,...
9375975,AAAAAAKE201612240,PAS,14621,1973,2DSD,GAS,False
9375976,,PAS,14120,1941,4DSD,GAS,False
9375977,,SRF,10580,1967,2DSD,GAS,False
9375978,,PAS,14047,1958,CONV,GAS,False


In [7]:
#Seperate Data by fuel_clas
elec_df = vehicle_data.loc[vehicle_data['elec_hybrid'] == True]
std_df = vehicle_data.loc[vehicle_data['elec_hybrid'] == False]

#Rename Columns
elec_df = elec_df.rename(columns={"elec_hybrid": "Electric"})
std_df = std_df.rename(columns={"elec_hybrid": "Standard"})

#remove unused columns
elec_df = elec_df.drop(columns =['vin','model_year', 'body_type', 'registration_class', 'model_year',])
std_df = std_df.drop(columns =['vin','model_year', 'body_type', 'registration_class', 'model_year',])

In [ ]:
#Merge the 2 data sets
combined_df = pd.merge(elec_df, std_df, on=['zip','zip'], how="left" )
combined_df

In [ ]:
# # #sort by zip and type
# fuel_df = pd.DataFrame(vehicle_data.groupby(['elec_hybrid','zip']).size())

# #fuel_df = fuel_df.drop(columns =['vin','registration_class', 'model_year',])

# fuel_df
 

In [ ]:
#Add AQI data
aqi_df = pd.read_csv("AQIndex2020.csv", index_col=1)
aqi_df

In [ ]:
#merge data based on zip

fuel_aqi_df = pd.merge(aqi_df, fuel_df, on=['zip','zip'], how="left" )
fuel_aqi_df